In [2]:
import numpy as np
import torch
from tqdm.notebook import tqdm

import utils
import datasets

In [3]:
dataset_sagittal = np.load("./data/tdsc/tumors_3d/z/signatures.npy")
dataset = datasets.Tumors(path="./data/tdsc/tumors_3d/z")
print(len(dataset))
print(dataset_sagittal.shape)

100
(100, 180, 360)


In [4]:
signature_dataset = []

for index, (v,m,y) in enumerate(dataset):
    sig = np.zeros([180, 360])
    sig = dataset_sagittal[index,:,:]
    sig = torch.tensor(sig)
    y = torch.tensor(y)
    signature_dataset.append((sig ,y))

In [5]:
train_loader = torch.utils.data.DataLoader(signature_dataset, batch_size=16, shuffle=True, pin_memory=True, num_workers=4)

In [6]:
class SequenceClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device='cpu'):
        super(SequenceClassifier, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size * 180, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [7]:
device = 'cuda'
num_epochs = 50
learning_rate = 10e-4
model = SequenceClassifier(360, 180, 1, 2, device=device).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
num_corrects_test = 0

for index in tqdm(range(len(signature_dataset))):
    train_data = signature_dataset[:index] + signature_dataset[index+1:]
    test_data = signature_dataset[index]
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True, pin_memory=True, num_workers=4)
    model = SequenceClassifier(360, 180, 1, 2, device=device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for data in train_loader:
            
            x,y = data
            x = x.float().to(device)
            y = y.to(device)
            
            predictions = model(x)
            loss = criterion(predictions, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()
        
        with torch.no_grad():
            num_corrects = 0
            for data in train_loader:
                x,y = data
                x = x.float().to(device)
                y = y.to(device)
                predictions = model(x)
                predictions = torch.argmax(predictions, 1)
                num_corrects += (y == predictions).sum()
                
            print(f"Epoch #{epoch} training acc is: {num_corrects/100:.2f}")
        model.train()
    
    model.eval()
    with torch.no_grad():
        x,y = test_data
        x = x.float().unsqueeze(0).to(device)
        y = y.unsqueeze(0).to(device)
        
        predictions = model(x)
        predictions = torch.argmax(predictions, 1)
        print(predictions)
        num_corrects_test += (predictions == y).sum()
    model.train()
    
print(f"Test accuracy is: {num_corrects_test/100:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch #0 training acc is: 0.48
Epoch #1 training acc is: 0.67
Epoch #2 training acc is: 0.72
Epoch #3 training acc is: 0.70
Epoch #4 training acc is: 0.62
Epoch #5 training acc is: 0.66
Epoch #6 training acc is: 0.71
Epoch #7 training acc is: 0.77
Epoch #8 training acc is: 0.76
Epoch #9 training acc is: 0.82
Epoch #10 training acc is: 0.78
Epoch #11 training acc is: 0.78
Epoch #12 training acc is: 0.79
Epoch #13 training acc is: 0.70
Epoch #14 training acc is: 0.74
Epoch #15 training acc is: 0.78
Epoch #16 training acc is: 0.79
Epoch #17 training acc is: 0.80
Epoch #18 training acc is: 0.80
Epoch #19 training acc is: 0.84
Epoch #20 training acc is: 0.82
Epoch #21 training acc is: 0.81
Epoch #22 training acc is: 0.82
Epoch #23 training acc is: 0.83
Epoch #24 training acc is: 0.85
Epoch #25 training acc is: 0.82
Epoch #26 training acc is: 0.82
Epoch #27 training acc is: 0.84
Epoch #28 training acc is: 0.82
Epoch #29 training acc is: 0.83
Epoch #30 training acc is: 0.86
Epoch #31 training